In [37]:
# Import necessary libraries
import numpy as np
import pandas as pd
import music21 as m21 # Used for music data processing, analysis, metadata extraction, etc. 
import mido as md # Used for MIDI file visualization and manipulation
import pretty_midi as pm 
import librosa as lb
import glob
import threading
import os
import fluidsynth as fs # For MIDI playback as MP3/WAV files
import matplotlib.pyplot as plt
from multiprocessing import Pool

In [2]:
df = pd.read_csv("Data\\Preprocessed\\file_info_processed.csv")

FilePaths = df["file_paths"]
SongNames = df["song_names"]
Artists = df["artists"]

df

,file_paths,song_names,artists
0,Data\Archive\.38 Special\Caught Up In You.mid,Caught Up In You,38 Special
1,Data\Archive\.38 Special\Fantasy Girl.mid,Fantasy Girl,38 Special
2,"Data\Archive\10,000_Maniacs\A_Campfire_Song.mid",A Campfire Song,"10,000 Maniacs"
3,Data\Archive\101_Strings\Theme_From_The_Godfat...,Theme From The Godfather,101 Strings
4,Data\Archive\10cc\Dreadlock_Holiday.1.mid,Dreadlock Holiday 1,10cc
...,...,...,...
17224,Data\Archive\ZZ_Top\Sleeping_Bag.mid,Sleeping Bag,ZZ Top
17225,Data\Archive\ZZ_Top\Tush.1.mid,Tush 1,ZZ Top
17226,Data\Archive\ZZ_Top\Tush.2.mid,Tush 2,ZZ Top
17227,Data\Archive\ZZ_Top\Tush.mid,Tush,ZZ Top


In [52]:
def get_metadata(midi_file):
    score = m21.converter.parse(midi_file)

    key = score.analyze("key")
    time_signature = score.getTimeSignatures()[0]
    tempo_mark = score.metronomeMarkBoundaries()[0]
    duration = score._getDuration()
    tempo = tempo_mark[2]

    return key, time_signature, duration, tempo

In [53]:
# Test metadata extraction
x = get_metadata(FilePaths[204])
print(x)

print(type(x))
print(type(x[0]))
print(type(x[1]))
print(type(x[2]))

(<music21.key.Key of C major>, <music21.meter.TimeSignature 4/4>, <music21.duration.Duration 1094/3>, <music21.tempo.MetronomeMark Quarter=115>)
<class 'tuple'>
<class 'music21.key.Key'>
<class 'music21.meter.base.TimeSignature'>
<class 'music21.duration.Duration'>
